### 우리가 모델링을 하는 이유는 예측 성능이 높은 고품질의 모델을 만드는 것이다. 

## 11.1 교차 검증 모델 만들기 : k-fold
* 모델 훈련 -> fitting -> k-fold로 검증하기

In [1]:
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

In [2]:
digits = datasets.load_digits() 
features = digits.data # 특성행렬
target = digits.target # 타깃벡터

standardizer = StandardScaler() #표준화 객체 만들기
logit = LogisticRegression() #로지스틱 회귀 객체 만들기

# 표준화한 다음 로지스틱 회귀를 실행하는 파이프라인 제작
pipeline = make_pipeline(standardizer, logit)

# k-fold 교차검증 만들기
kf = KFold(n_splits = 10, shuffle =True, random_state = 1)

# k-fold 교차검증 수행
cv_results = cross_val_score(pipeline, #파이프라인
                            features, #특성행렬
                            target, #타깃벡터
                            cv = kf, #교차검증 기법 
                            scoring = 'accuracy', #평가 지표
                            n_jobs = -1) #모든 CPU 코어 사용 - 작업 속도 높임

# 평균 계산
cv_results.mean()

0.964931719428926

### 이전에 본 적 없는 데이터에서 얼마나 좋은 예측을 만들어낼 수 있는지 알 수 있기 위하여
* 데이터의 일부를 테스트용으로 떼어둔다 = hold-out, validation(검증)

#### 방법 1. Test_set, Training_set 으로 나누기
* 검증에서 샘플은 training_set, test_set으로 나뉜다
* training_set와 타겟벡터를 활용해서 최선의 예측을 만드는 방법을 모델 훈련으로 가르친다
* test_set에서 모델의 성능을 평가한다
* 한계 
    * 모델 성능은 테스트 세트로 나뉜 일부 샘플에 의해 결정된다
    * 전체 가용 데이터를 사용하여 모델을 훈련하고 테스트할 수 없다
#### 방법 2. K-Fold cross validation (KFCV)
* 데이터를 fold라고 부르는 K개의 부분으로 나눈다
* K-1개를 합쳐 하나의 training_set으로 모델 훈련 후 나머지 1개를 test_set으로 사용한다
* 이 과정을 K번 반복하여서 모델 성능을 평균하여 최종 성능을 산출함

### KFold 결과값
* cv_results : 10번의 test_set 성능 산출개별값 확인 가능

### KFold 유의사항
* 각 샘플이 다른 샘플과 독립적으로 생성되었다고 가정 ( Independent Identity Distributed )
    * IID일때는 shuffle해준다 (why?) 
* 분류기를 평가할 경우 -> stratified k-fold
    * KFold - >  StratifiedKFold
    샘플 비중 일치시키기
* 전처리 최종본을 test,train으로 나눠야 한다
    * 편하게 해주는 도구 : pipeline

### KFold의 주요한 매개변수들
* cv = kf
* scoring = 'accuracy'
    모델성공의 측정 방법을 결정
* n_jobs = -1
    모든 CPU 코어 사용 -> 연산속도
    
### 기타
* LOOCV : 폴드의 수가 k의 샘플의 개수와 같다 (LeaveOneOut)클래스 = KFold(n_splits=n) #n=샘플개수
* ShuffleSplit : 반복횟수에 상관없이 훈련 폴드와 테스트 폴드 크기를 임의로 지정할 수 있음
* StratifiedShuffleSplit : 계층별로 교차검증  
교차검증 반복실행
* RepeatedKFold
* StratifiedRepeatedKFOld

* 분할기를 만들고 -> cv 파라미터값을 다르게 설정해주면 됨


In [4]:
from sklearn.model_selection import train_test_split
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size = 0.1, random_state = 1)
standardizer.fit(features_train)

features_train_std = standardizer.transform(features_train)
features_test_std = standardizer.transform(features_test)

In [7]:
features_train_std.shape

(1617, 64)

In [8]:
features.shape

(1797, 64)

### 11.2 기본 회귀 모델 만들기
* DummyRegressor : 비교용으로 단순하게 만드는 것. 실제 문제 상황에서 사용할 거는 아님..

In [30]:
from sklearn.datasets import load_boston
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split

boston = load_boston()

features,target = boston.data, boston.target

features_train, features_test, target_train, target_test = train_test_split(
    features, target, random_state = 1) # seed넘버

dummy = DummyRegressor(strategy = 'mean') # 'constant'를 넣으면 모든 샘플에 대해 일정한 값으로 예측하는 더미 회귀 모델을 만들 수 있음
dummy.fit(features_train, target_train)
dummy.score(features_test, target_test) # R squared

-0.005676907086416438

In [31]:
from sklearn.linear_model import LinearRegression
ols = LinearRegression()
ols.fit(features_train, target_train)
ols.score(features_test, target_test)

0.7789410172622865

#### R squared
* 결정계수
* 설명력
* 1에 가까울수록 타깃벡터의 분산을 잘 설명함

#### DummyRegressor 파라미터
* 'constant'
* 'mean'
* 'median'
* 'quantile', quantile = 1.0 # max값

## 11.3 기본 분류 모델 만들기

In [15]:
from sklearn.datasets import load_iris
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split

iris = load_iris()
features, target = iris.data, iris.target
features_train, features_test, target_train, target_test = train_test_split(
    features, target, random_state =0) #seed넘버

dummy = DummyClassifier(strategy = 'uniform', random_state = 1)

dummy.fit(features_train, target_train)

dummy.score(features_test, target_test)

0.42105263157894735

In [17]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier()
classifier.fit(features_train, target_train)
classifier.score(features_test, target_test)

/Users/yeonjin/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9736842105263158

### 분류 모델의 성능을 측정하는 일반적인 방법은, 랜덤한 추측보다 얼마나 더 나은지를 비교하는 것이다

* strategy
    * uniform
    * stratified
    * most_frequent : 훈련 세트 타깃에서 가장 많은 값으로 예측한다

In [19]:
import numpy as np
np.bincount(target_train)

array([37, 34, 41])

In [21]:
dummy = DummyClassifier(strategy = 'most_frequent')
dummy.fit(features_train, target_train)
dummy.predict(features_test)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])